In [1]:
from collections import Counter, defaultdict
from functools import partial
import math, random

In [2]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]


In [3]:
def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """ 입력된 Attribute를 기준으로 데이터를 나눠서 출력해줌 (attribute_dict, label)"""

    return group_by(inputs, lambda x: x[0][attribute])

partition_by(inputs, "level")["Junior"]

[({'lang': 'Python', 'level': 'Junior', 'phd': 'no', 'tweets': 'no'}, True),
 ({'lang': 'R', 'level': 'Junior', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'R', 'level': 'Junior', 'phd': 'yes', 'tweets': 'yes'}, False),
 ({'lang': 'Python', 'level': 'Junior', 'phd': 'no', 'tweets': 'yes'}, True),
 ({'lang': 'Python', 'level': 'Junior', 'phd': 'yes', 'tweets': 'no'}, False)]

In [4]:
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

In [5]:
level_junior_partition = partition_by(inputs, "level")
data_entropy(level_junior_partition["Junior"])

0.9709505944546686

In [6]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets) # 하나의 Attribute에 나타나는 각 경우의 합을 구함
    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

In [7]:
level_junior_partition = partition_by(inputs, "level")
print(level_junior_partition)

defaultdict(<class 'list'>, {'Senior': [({'tweets': 'no', 'phd': 'no', 'level': 'Senior', 'lang': 'Java'}, False), ({'tweets': 'no', 'phd': 'yes', 'level': 'Senior', 'lang': 'Java'}, False), ({'tweets': 'no', 'phd': 'no', 'level': 'Senior', 'lang': 'Python'}, False), ({'tweets': 'yes', 'phd': 'no', 'level': 'Senior', 'lang': 'R'}, True), ({'tweets': 'yes', 'phd': 'yes', 'level': 'Senior', 'lang': 'Python'}, True)], 'Mid': [({'tweets': 'no', 'phd': 'no', 'level': 'Mid', 'lang': 'Python'}, True), ({'tweets': 'yes', 'phd': 'yes', 'level': 'Mid', 'lang': 'R'}, True), ({'tweets': 'no', 'phd': 'yes', 'level': 'Mid', 'lang': 'Python'}, True), ({'tweets': 'yes', 'phd': 'no', 'level': 'Mid', 'lang': 'Java'}, True)], 'Junior': [({'tweets': 'no', 'phd': 'no', 'level': 'Junior', 'lang': 'Python'}, True), ({'tweets': 'yes', 'phd': 'no', 'level': 'Junior', 'lang': 'R'}, True), ({'tweets': 'yes', 'phd': 'yes', 'level': 'Junior', 'lang': 'R'}, False), ({'tweets': 'yes', 'phd': 'no', 'level': 'Junior',

In [8]:
partition_entropy(level_junior_partition.values())

0.6935361388961919

In [9]:
def partition_entropy_by(inputs,attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [10]:
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [11]:
def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf

    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf

    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))



    partitions = partition_by(inputs, best_attribute)

    new_candidates = [a for a in split_candidates
                      if a != best_attribute]
    

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

In [12]:
print("building the tree")
tree = build_tree_id3(inputs)
print(tree)

building the tree
('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), None: True, 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True})})


In [13]:
def classify(tree, input):
    """classify the input using the given decision tree"""

    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree

    # otherwise find the correct subtree
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # None if input is missing attribute
    print(subtree_key)

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree

    subtree = subtree_dict[subtree_key] # choose the appropriate subtree

    return classify(subtree, input)     # and use it to classify the input

In [14]:
print("Junior / Java / tweets / no phd", classify(tree,
        { "level" : "Junior",
          "lang" : "Java",
          "tweets" : "yes",
          "phd" : "no"} ))

Junior
no
Junior / Java / tweets / no phd True


In [15]:
print("Junior / Java / tweets / phd", classify(tree,
        { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} ))

print("Intern", classify(tree, { "level" : "Intern" } ))
print("Senior", classify(tree, { "level" : "Senior" } ))

Junior
yes
Junior / Java / tweets / phd False
Intern
Intern True
Senior
None
Senior False


In [16]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]